# Trabalho Análise de Dados

# Ferramentas de Softwares para uso em Ciência de Dados II

# Proferssor Marcos Guimarães

In [ ]:
# Importação das bibliotecas genéricas mais usadas
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from wordcloud import WordCloud, STOPWORDS
from IPython.display import Image, HTML

### Importação dos dados

Importação da base de filmes do *The Movies DataSet*, projeto *opensource* presente no projeto do kaggle, disponível em:
[https://www.kaggle.com/rounakbanik/the-movies-dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset)

A base foi incorporada ao projeto e está presente na pasta ```../input```, de onse se fará a importação.

In [ ]:
# Importando dados
df_filmes = pd.read_csv('../input/movies_metadata.csv', low_memory=False)

# Listando as 5 primeiras linhas do arquivo importado
df_filmes.head()

In [ ]:
# Exibindo informações do arquivo importado
df_filmes.info()

In [ ]:
# Limpeza de colunas desnecessárias
df_filmes = df_filmes.drop(['imdb_id', 'homepage', 'original_title'], axis=1)

# Criação da coluna ano para utilização posterior
# Utilizando o campo release_date
df_filmes['ano'] = pd.to_datetime(df_filmes['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

## Análise Exploratória

### Tempo de duraçao dos filmes

In [ ]:
df_filmes['runtime'].describe()

A média de duraçao dos filmes é de aproximadamente 1 hora e 30 minutos, sem muita variação, uma vez que o terceiro quartil conta com 1 hora e 47 minutos. O maior tempo presente na base é de 1256 minutes (20 horas).

In [ ]:
# Converão para float
df_filmes['runtime'] = df_filmes['runtime'].astype('float')

# Plotando o gráfico
plt.figure(figsize=(12,6))
sns.distplot(df_filmes[(df_filmes['runtime'] < 300) & (df_filmes['runtime'] > 0)]['runtime'])
plt.xlabel('Duração em minutos')
plt.ylabel("Quantidade de filmes")
plt.title("Distribuição da duraçao dos filmes")


A distribuição do tempo do filme em minutos pela quantidade total de filmes mostra uma concetrração grande de filmes que possuem duração entre 80 a 120 minutos.

### Filmes por país produtor

In [ ]:
# Tratamento de Dado
# Transformação da coluna production_countries que é um json, para extrair o país produtos
df_filmes['production_countries'] = df_filmes['production_countries'].fillna('[]').apply(ast.literal_eval)
df_filmes['production_countries'] = df_filmes['production_countries'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

s = df_filmes.apply(lambda x: pd.Series(x['production_countries']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'countries'

# Criação de novo Data Set para armazenar a quantidade de filmes por país, agrupando e contando.
con_df = df_filmes.drop('production_countries', axis=1).join(s)
con_df = pd.DataFrame(con_df['countries'].value_counts())
con_df['country'] = con_df.index
con_df.columns = ['Quantidade', 'Pais']
con_df = con_df.reset_index().drop('index', axis=1)

# Listando o TOP 20
con_df.head(20)

Lista dos filmes agrupados pelo seu país produtor em ordem descrescente.

In [ ]:
# Criação do plot horizontal
sns.set(style="whitegrid")
plt.subplots(figsize=(8, 15))
sns.set_color_codes("pastel")
sns.barplot(x="Quantidade", y="Pais", data=con_df.head(20),
            label="Total", color="b")
plt.xlabel('Quantidade de filmes produzidos')
plt.ylabel("País")
plt.title("TOP 20 filmes por país produtor")

Analisando ográfico acima, percebe-se uma clara hegemonia dos filmes feitos nos Estados Unidos diante dos outros países. O segundo colocado, Reino Unido, não representa nem um terço do primeiro colocado.

## Nome do Filme

In [ ]:
# Junta todos os nomes em inglês para a variável todos_titulos
df_filmes['title'] = df_filmes['title'].astype('str')
todos_titulos = ' '.join(df_filmes['title'])

# Plota o wordcloud
title_wordcloud = WordCloud(stopwords=STOPWORDS, background_color='white', height=2000, width=4000).generate(todos_titulos)
plt.figure(figsize=(16,8))
plt.imshow(title_wordcloud)
plt.axis('off')
plt.show()

Geração de um wordcloud. Todos os títulos em inglês foram adicionados a variável ```todos_titulos```  após foi trtansformado num Wordcloud onde é possível visualizar a incidência das palavras com mais utilizações nos nomes de filmes.

A pavra *LOVE* é a mais usada, seguido de perto de *MAN* e *GIRL*.

### Linguagem original do filme

In [ ]:
# Dropa as linguagens duplicadas 
df_filmes['original_language'].drop_duplicates().shape[0]

Na base analisada há 93 linguagens de filems.

In [ ]:
# Agrupa por idioma (original_language) e conta a quantidade de filmes por idioma
idioma = pd.DataFrame(df_filmes['original_language'].value_counts())
idioma['language'] = idioma.index
idioma.columns = ['Quantidade', 'Idioma']

# Lista os 20 idiomas mais frequentes
idioma.head(20)

O inglês é o idioma predominante, seguido pelo francês, em razão da quantidade de paises falados que possuem relevância na quantidade de países produtores (França + Canada).

O português aparece na 12º colocação.

### Gênero

In [ ]:
# Transformação do gênero uma vez que o armazenado é um JSON
df_filmes['genres'] = df_filmes['genres'].fillna('[]').apply(ast.literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
s = df_filmes.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'

In [ ]:
# Criar o generos_df para contagem e classificação
generos_df = df_filmes.drop('genres', axis=1).join(s)
generos_df['genre'].value_counts().shape[0]

Existem 32 gêneros de filmes classificados.

In [ ]:
# Pega os 20 principais gêneros e conta quantos filmes possuem nesses gêneros
generos_populares = pd.DataFrame(generos_df['genre'].value_counts()).reset_index()
generos_populares.columns = ['Genero', 'Filmes']

# Lista os 20 gêneros mais frequentes
generos_populares.head(20)

Lista dos 20 gêneros com maior quantidade de filmes

In [ ]:
# Plota filmes por gênero num barplot vertical
plt.figure(figsize=(28,10))
sns.barplot(x='Genero', y='Filmes', data=generos_populares.head(20))
plt.xlabel("Gênero")
plt.ylabel("Quantidade")
plt.title("Quantidade de filmes por gênero - TOP 20")
plt.show()

Podemos verificar visualmente que os gêneros Drama e Comédia são os mais populares.

### Filmes com maiores receitas

In [ ]:
# Arrumando o endereço da imagem do poster, acrescentando os parêmtros HTML
base_poster_url = 'http://image.tmdb.org/t/p/w185/'
df_filmes['poster_path'] = "<img src='" + base_poster_url + df_filmes['poster_path'] + "' style='height:100px;'>"

# Gera a lista de filmes, ordenando por maior receita e as 10 maiores arrecadaçoes
maiores_receitas = df_filmes[['poster_path', 'title', 'revenue', 'ano']].sort_values('revenue', ascending=False).head(10)

# Transforma em bilhões de dolares a escala para ficar mais claro a análise
maiores_receitas['revenue'] = maiores_receitas['revenue'] / 1000000000
maiores_receitas.columns = ['Poster', 'Titulo', 'Receitas (bilhões)', 'Ano']
pd.set_option('display.max_colwidth', 100)

# Exibe o poster utilizando o endereço da imagem armazenada no dataset
HTML(maiores_receitas.to_html(escape=False))

Acima vemos os 10 filmes com as maiores receitas de todos os tempos, ordenados por receita, e exibindo o seu poster.

In [ ]:
# Gera a lista de filmes, ordenando por maior receita
maiores_receitas_ptbr = df_filmes[['poster_path', 'title', 'revenue', 'ano', 'original_language']]

# Filtra os filmes com idioma português e as 20 maiores arrecadaçoes
maiores_receitas_ptbr = maiores_receitas_ptbr[maiores_receitas_ptbr['original_language'] == 'pt' ].sort_values('revenue', ascending=False).head(20)

# Transforma em milhões de dolares a escala para ficar mais claro a análise
maiores_receitas_ptbr['revenue'] = maiores_receitas_ptbr['revenue'] / 1000000
maiores_receitas_ptbr.columns = ['Poster', 'Titulo', 'Receita (milhões dolares)', 'Ano', 'Idioma']
pd.set_option('display.max_colwidth', 100)

# Exibe o poster utilizando o endereço da imagem armazenada no dataset
HTML(maiores_receitas_ptbr.to_html(escape=False))

As 10 maiores arrecadações de filmes em português, aqui contando na lista alguns em português de Portugal, entretanto o maior de todos, é o brasileiro **Tropa de Elite** (*Elite Squad: The Enemy Within*).